# E-Commerce Shipping Data

### Product Shipment Delivered on time or not? To Meet E-Commerce Customer Demand

![image.png](../Images/Shipment.png)

e-commerce 배송의 정시 도착여부 ( 1 : 정시배송, 0 : 정시미배송 )를 판단하시오. <br>
단, 두 개 이상의 모형의 성능을 비교하여 가장 우수한 모형을 선정할 것

### Library & Data Import

In [1]:
import pandas as pd
import numpy as np

In [2]:
X_test = pd.read_csv('../Datasets/Shipment_X_test.csv')
X_train = pd.read_csv('../Datasets/Shipment_X_train.csv')
y_train = pd.read_csv('../Datasets/Shipment_y_train.csv')

### 1. Data Exploration

In [3]:
X_test

,ID,Warehouse_block,Mode_of_Shipment,Customer_care_calls,Customer_rating,Cost_of_the_Product,Prior_purchases,Product_importance,Gender,Discount_offered,Weight_in_gms
0,6811,D,Ship,5,2,259,5,low,F,7,1032
1,4320,F,Ship,3,5,133,3,medium,F,4,5902
2,5732,F,Road,3,4,191,5,medium,F,4,4243
3,7429,D,Ship,4,2,221,3,low,M,10,4126
4,2191,D,Flight,4,5,230,2,low,F,38,2890
...,...,...,...,...,...,...,...,...,...,...,...
4396,2610,F,Flight,4,1,157,3,medium,M,31,1712
4397,3406,B,Road,3,5,139,2,medium,M,7,5536
4398,10395,A,Road,4,1,170,10,medium,F,3,5211
4399,3646,B,Ship,3,1,244,3,medium,F,1,5695


In [4]:
X_train

,ID,Warehouse_block,Mode_of_Shipment,Customer_care_calls,Customer_rating,Cost_of_the_Product,Prior_purchases,Product_importance,Gender,Discount_offered,Weight_in_gms
0,6045,A,Flight,4,3,266,5,high,F,5,1590
1,44,F,Ship,3,1,174,2,low,M,44,1556
2,7940,F,Road,4,1,154,10,high,M,10,5674
3,1596,F,Ship,4,3,158,3,medium,F,27,1207
4,4395,A,Flight,5,3,175,3,low,M,7,4833
...,...,...,...,...,...,...,...,...,...,...,...
6593,8610,F,Road,5,2,221,6,medium,M,4,1952
6594,3390,F,Ship,4,5,256,3,medium,M,10,4504
6595,5150,F,Ship,3,1,217,4,medium,F,1,5761
6596,3950,F,Road,4,5,174,3,medium,F,8,5576


In [5]:
y_train

,ID,Reached.on.Time_Y.N
0,6045,0
1,44,1
2,7940,1
3,1596,1
4,4395,1
...,...,...
6593,8610,1
6594,3390,1
6595,5150,1
6596,3950,0


In [6]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6598 entries, 0 to 6597
Data columns (total 11 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   ID                   6598 non-null   int64 
 1   Warehouse_block      6598 non-null   object
 2   Mode_of_Shipment     6598 non-null   object
 3   Customer_care_calls  6598 non-null   object
 4   Customer_rating      6598 non-null   int64 
 5   Cost_of_the_Product  6598 non-null   int64 
 6   Prior_purchases      6598 non-null   int64 
 7   Product_importance   6598 non-null   object
 8   Gender               6598 non-null   object
 9   Discount_offered     6598 non-null   int64 
 10  Weight_in_gms        6598 non-null   int64 
dtypes: int64(6), object(5)
memory usage: 567.1+ KB


In [7]:
X_train.corr()

,ID,Customer_rating,Cost_of_the_Product,Prior_purchases,Discount_offered,Weight_in_gms
ID,1.000000,0.001824,0.204097,0.155189,-0.589201,0.276868
Customer_rating,0.001824,1.000000,0.007704,0.013940,-0.004975,-0.007009
Cost_of_the_Product,0.204097,0.007704,1.000000,0.127362,-0.147798,-0.139016
Prior_purchases,0.155189,0.013940,0.127362,1.000000,-0.093055,-0.168903
Discount_offered,-0.589201,-0.004975,-0.147798,-0.093055,1.000000,-0.370676
Weight_in_gms,0.276868,-0.007009,-0.139016,-0.168903,-0.370676,1.000000


### 2. Data Preprocessing

#### (1) Missing Value

결측값 없음

### 3. Data Modeling

#### (1) One-Hot Encoding

In [14]:
from sklearn.preprocessing import OneHotEncoder

X_train_cat = X_train.select_dtypes('object').copy()
X_test_cat =  X_test.select_dtypes('object').copy()

ohe = OneHotEncoder(sparse=False)

ohe.fit(X_train_cat)

X_train_ohe = ohe.transform(X_train_cat)
X_test_ohe = ohe.transform(X_test_cat)

#### (2) Scaling

In [20]:
from sklearn.preprocessing import MinMaxScaler

X_train_num = X_train.select_dtypes(exclude='object').copy()
X_test_num = X_test.select_dtypes(exclude='object').copy()

scaler = MinMaxScaler()

scaler.fit(X_train_num)

X_train_sca = scaler.transform(X_train_num)
X_test_sca = scaler.transform(X_test_num)

#### (3) Data Split

In [22]:
X_TRAIN = np.concatenate([X_train_ohe, X_train_sca], axis=1)
X_TEST = np.concatenate([X_test_ohe, X_test_sca], axis=1)

y_TRAIN = y_train['Reached.on.Time_Y.N']

print(type(X_TRAIN), type(X_TEST), type(y_TRAIN))
print(X_TRAIN.shape, X_TEST.shape, y_TRAIN.shape)

<class 'numpy.ndarray'> <class 'numpy.ndarray'> <class 'pandas.core.series.Series'>
(6598, 19) (4401, 19) (6598,)


In [24]:
from sklearn.model_selection import train_test_split

xtrain, xtest, ytrain, ytest = train_test_split(X_TRAIN, y_TRAIN, test_size = 0.25, stratify=y_fin_train, random_state=1234)

print(xtrain.shape, xtest.shape, ytrain.shape, ytest.shape)

(4948, 19) (1650, 19) (4948,) (1650,)


 ### 4. Modeling

In [25]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

In [26]:
def make_models(xtrain, xtest, ytrain, ytest):
    model1 = LogisticRegression().fit(xtrain, ytrain)
    print('model1', get_scores(model1, xtrain, xtest, ytrain, ytest))

    model2 = DecisionTreeClassifier(random_state=0).fit(xtrain, ytrain)
    print('model2', get_scores(model2, xtrain, xtest, ytrain, ytest))

    for d in range(3, 8):
        model2 = DecisionTreeClassifier(max_depth=d, random_state=0).fit(xtrain, ytrain)
        print('model2', d, get_scores(model2, xtrain, xtest, ytrain, ytest))

    model3 = RandomForestClassifier(random_state=0).fit(xtrain, ytrain)
    print('model3', get_scores(model3, xtrain, xtest, ytrain, ytest))

    for d in range(3, 8):
        model3 = RandomForestClassifier(500, max_depth=d, random_state=0).fit(xtrain, ytrain)
        print('model3', d, get_scores(model3, xtrain, xtest, ytrain, ytest))

    model4 = XGBClassifier(eval_metric='logloss').fit(xtrain, ytrain)
    print('model4', get_scores(model4, xtrain, xtest, ytrain, ytest))

### 5. Model Evaluation

In [27]:
from sklearn.metrics import roc_auc_score

def get_scores(model, xtrain, xtest, ytrain, ytest):
    A = model.score(xtrain, ytrain)
    
    ypred = model.predict_proba(xtest)[:, 1]
    
    B = roc_auc_score(ytest, ypred)
    
    return f'{A:.4} {B:.4}'

In [28]:
make_models(xtrain, xtest, ytrain, ytest)

model1 0.6586 0.7508
model2 1.0 0.6258
model2 3 0.6952 0.7407
model2 4 0.7029 0.7398
model2 5 0.7061 0.7479
model2 6 0.7142 0.7338
model2 7 0.7197 0.7394
model3 1.0 0.7481
model3 3 0.6983 0.7522
model3 4 0.6987 0.7516
model3 5 0.7055 0.7513
model3 6 0.716 0.7488
model3 7 0.7411 0.7507
model4 0.961 0.738


In [29]:
final_model = RandomForestClassifier(max_depth=4, random_state=0).fit(xtrain, ytrain)

print('final model', get_scores(final_model, xtrain, xtest, ytrain, ytest))

final model 0.7039 0.7504


### 6. Save Result

In [31]:
y_pred = final_model.predict_proba(X_TEST)[:, 1]

result = pd.DataFrame({"y_pred" : y_pred})
result.to_csv("./on_time_y_pred.csv")